# Hugging Face Upload & Inference Demo

This notebook demonstrates how to:
1. Upload your fine-tuned model weights to Hugging Face Hub.
2. Download the weights from the Hub.
3. Run inference using the downloaded weights.
4. Save output images to a new folder in the `notebooks` directory.

In [ ]:
# Install required libraries if not already installed
!pip install huggingface_hub diffusers transformers safetensors torch --quiet

In [ ]:
from huggingface_hub import HfApi, HfFolder, upload_file, notebook_login
import os

# --- User Variables ---
repo_id = 'TheoMefff/flux_schnell_baroque'  # Change if needed
local_model_path = '../output/flux_schnell/flux_schnell_000002000.safetensors'  # Update if needed
destination_name = os.path.basename(local_model_path)

# Authenticate with your Hugging Face token
notebook_login()

In [ ]:
# Upload the model weights to the Hugging Face Hub
api = HfApi()
token = HfFolder.get_token()
api.upload_file(
    path_or_fileobj=local_model_path,
    path_in_repo=destination_name,
    repo_id=repo_id,
    repo_type="model",
    token=token
)
print(f'Uploaded {destination_name} to {repo_id}')

---
## Download from Hugging Face and Run Inference
We'll use the `diffusers` library to load the model and generate images.

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from huggingface_hub import snapshot_download
import os
from PIL import Image

# Download model snapshot from Hugging Face
model_dir = snapshot_download(repo_id, repo_type="model")
print(f'Model downloaded to: {model_dir}')

# Create output folder inside notebooks/
notebook_dir = os.path.dirname(os.path.abspath("__file__")) if '__file__' in globals() else os.getcwd()
output_dir = os.path.join(notebook_dir, 'inference_outputs')
os.makedirs(output_dir, exist_ok=True)

# Load pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    model_dir,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
if torch.cuda.is_available():
    pipe = pipe.to("cuda")

# Prompts for inference
prompts = [
    "a beautiful painting of a sunset",
    "a futuristic cityscape at night",
    "a portrait of a woman in renaissance style"
]

for idx, prompt in enumerate(prompts):
    image = pipe(prompt).images[0]
    out_path = os.path.join(output_dir, f'output_{idx+1}.png')
    image.save(out_path)
    print(f'Saved: {out_path}')